In [1]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING --progress_bar None --disable_caching veff_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s veff_Scene"

Manim Community v0.18.0.post0

Jupyter Capture Output v0.0.11


In [150]:
class EffectivePotential(Mobject):
    def __init__(self, center, x_range, y_range, x_length, y_length, alpha, mass, E, L, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.x_range = x_range
        self.y_range = y_range
        self.x_length = x_length
        self.y_length = y_length
        self.alpha = alpha
        self.mass = mass
        self.E = E
        self.L = L

        # coordinate system
        self.ax = Axes(
            x_range = self.x_range, y_range = self.y_range, x_length = self.x_length, y_length = self.y_length, 
            axis_config = {"stroke_color": BLACK, "tip_width": 0.15, "tip_height": 0.15, "include_ticks": False},
            ).move_to(self.center).set_opacity(0.5)
        self.ax_xlabel = self.ax.get_x_axis_label(Tex(r"$r$", font_size = 28, color = BLACK)).shift(0.1 *LEFT)
        self.ax_ylabel = self.ax.get_y_axis_label(Tex(r"$V$", font_size = 28, color = BLACK)).shift(0.15 * DOWN)
        self.add(self.ax, self.ax_xlabel, self.ax_ylabel)


    # returns the function for the central potential
    def central_potential(self, r):
        return -self.alpha / r
    

    # returns the effective potential for given angular momentum L
    def effective_potential(self, r):
        V = self.central_potential(r)
        return V + self.L**2 / (2*self.mass*r**2)
    

    # returns the minimum of the effective potential
    def get_mimimum(self):
        return self.L**2 / (self.mass*self.alpha)
    

    # returns the minimal radius as the initial condition
    def get_init_state(self):
        r = 0.1
        E_eff = self.effective_potential(r)
        while (E_eff > self.E):
            r += 0.001
            E_eff = self.effective_potential(r)
        return np.array([r, 0])
    

    # returns the next state of a given state in the effective potential (velocity verlet)
    def get_next_state(self, state):
        # algorithm control parameters
        delta_r = 0.001
        delta_t = 0.02

        r = state[0]
        d_r = state[1]
        dd_r = (self.effective_potential(r-delta_r) - self.effective_potential(r+delta_r)) / (self.mass * 2*delta_r)

        a1 = np.array([d_r, dd_r])
        state[0] += a1[0]*delta_t + a1[1]*delta_t**2 / 2

        r = state[0]
        dd_r = (self.effective_potential(r-delta_r) - self.effective_potential(r+delta_r)) / (self.mass * 2*delta_r)

        a2 = np.array([d_r, dd_r])
        state[1] += (a1[1] + a2[1]) * delta_t / 2
        return state
    

    # returns the effective potential plot
    def draw_effective_potential(self):
        x = 0.1
        y = self.effective_potential(x)
        y_max = self.y_range[1]
        while (y > y_max):
            x += 0.005
            y = self.effective_potential(x)
        ax_plot = self.ax.plot(self.effective_potential, x_range = [x, self.x_range[1]], color = BLACK)
        ax_descriptor = Tex(r"$V_\mathrm{eff}$", font_size = 28, color = BLACK).move_to(self.ax.c2p(x, y, 0)).shift(0.5*DOWN + 0.5*RIGHT)
        return VGroup(ax_plot, ax_descriptor)
    

    # draws the line for the total energy of the system
    def draw_energy(self):
        ax_plot = self.ax.plot(lambda x: self.E, x_range = [-0.1, self.x_range[1]], color = RED).set_opacity(0.5)
        ax_descriptor = Tex(r"$E$", font_size = 28, color = RED).move_to(self.ax.c2p(-0.5, self.E, 0)).set_opacity(0.75)
        return VGroup(ax_plot, ax_descriptor)
    

    # draws the point on the potential for a given state
    def draw_point(self, state):
        r = state[0]
        veff_r = self.effective_potential(r)
        state_circle = Circle(radius = 0.1, color = RED, fill_opacity = 0.75).move_to(self.ax.c2p(r, veff_r, 0))
        return state_circle
    


    


class ConicTrajectory(Mobject):
    def __init__(self, center, x_range, y_range, x_length, y_length, alpha, mass, E, L, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.x_range = x_range
        self.y_range = y_range
        self.x_length = x_length
        self.y_length = y_length
        self.alpha = alpha
        self.mass = mass
        self.p = L**2/(alpha*mass)
        self.epsilon = np.sqrt( (2*E*L**2) / (alpha**2*mass) )

        # coordinate system
        self.ax = Axes(
            x_range = self.x_range, y_range = self.y_range, x_length = self.x_length, y_length = self.y_length, 
            axis_config = {"stroke_color": BLACK, "tip_width": 0.15, "tip_height": 0.15, "include_ticks": False},
            ).move_to(self.center).set_opacity(0.5)
        # self.add(self.ax)
        sun = Dot(point = self.ax.c2p(0, 0, 0), color = BLACK, radius = 0.1)
        self.add(sun)


    # analytical solution r(phi) of the 2-body-problem
    def get_r(self, phi):
        return self.p / (1 + self.epsilon*np.cos(phi))
    

    # reverse function phi(r) of the analytical 2-body-problem solution
    def get_phi(self, r):
        return np.arccos( (self.p/r - 1) / self.epsilon)


    # draw r(phi) analytical solution
    def draw(self):
        phi_array = np.linspace(0, 2*PI, 1000)
        r_array = np.array([self.get_r(phi) for phi in phi_array])

        x_vals = [r_array[i] * np.cos(phi_array[i]) for i in range(len(phi_array))]
        y_vals = [r_array[i] * np.sin(phi_array[i]) for i in range(len(phi_array))]

        r_phi_plot = self.ax.plot_line_graph(x_vals, y_vals, stroke_color = RED, add_vertex_dots = False, stroke_opacity = 0.5, stroke_width = 4)
        return r_phi_plot

In [151]:
%%capture_video --path "animations/effective_potential/effective_potential.mp4"
%%manim -qm --fps 60 $video_scene


class veff_Scene(ThreeDScene):
    def construct(self):
        self.camera.background_color = WHITE

        # physical parameters
        alpha = 13
        mass = 1
        L = 4
        E = -3

        # coordinate systems parameters
        veff_center = np.array([4, 0, 0])
        veff_x_range = [0, 6, 1]
        veff_y_range = [-6, 4, 1]
        veff_x_length = 5
        veff_y_length = 5

        effective_potential = EffectivePotential(veff_center, veff_x_range, veff_y_range, veff_x_length, veff_y_length, alpha, mass, E, L)
        self.add(effective_potential)

        effective_potential_plot = effective_potential.draw_effective_potential()
        self.add(effective_potential_plot)

        effective_potential_energy_plot = effective_potential.draw_energy()
        self.add(effective_potential_energy_plot)

        init_state = effective_potential.get_init_state()
        state_circle = effective_potential.draw_point(init_state)
        state_circle.getter = effective_potential.draw_point
        state_circle.state = init_state
        state_circle.state_getter = effective_potential.get_next_state
        self.add(state_circle)


        # polar trajectory coordinate system
        traj_center = np.array([-2, 0, 0])
        traj_x_range = [-4, 4, 1]
        traj_y_range = [-4, 4, 1]
        traj_x_length = 6
        traj_y_length = 6

        conic_trajectory = ConicTrajectory(traj_center, traj_x_range, traj_y_range, traj_x_length, traj_y_length, alpha, mass, -E, L)
        self.add(conic_trajectory)

        conic_trajectory_trajectory = conic_trajectory.draw()
        self.add(conic_trajectory_trajectory)



        # updater
        def eff_pot_updater(dot):
            state = dot.state
            new_state = dot.state_getter(state)
            dot_getter = dot.getter
            new_dot = dot_getter(new_state)
            dot.become(new_dot)


        t_tracker = ValueTracker(0)
        self.wait(1.5)
        state_circle.add_updater(eff_pot_updater)
        self.play(t_tracker.animate.set_value(5), rate_func= linear, run_time = 15)
        self.wait(3)
        

Output saved by overwring previous file at animations/effective_potential/effective_potential.mp4.
